In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.ensemble import  RandomForestClassifier
import pickle
import os

In [2]:
flight_data = (
    pd.read_csv('final_dataset_cleaned_v1.csv')
)

In [3]:
flight_data.shape

(12658696, 29)

In [4]:
flight_data.head(n = 3)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,TAIL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,...,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,coco,target
0,2020,1,1,1,3,2020-01-01,N951WN,ONT,"Ontario, CA",SFO,...,2020-01-01 18:00:00,12.2,-7.3,25.0,0.0,330.0,18.4,1017.3,3.0,1
1,2020,1,1,1,3,2020-01-01,N467WN,ONT,"Ontario, CA",SFO,...,2020-01-01 11:00:00,3.9,-3.4,59.0,0.0,210.0,5.4,1017.6,2.0,0
2,2020,1,1,1,3,2020-01-01,N7885A,ONT,"Ontario, CA",SJC,...,2020-01-01 20:00:00,12.8,-7.3,24.0,0.0,290.0,11.2,1015.8,2.0,0


In [5]:
flight_data['aircraft_type_new'] = [i.split('-')[0] for i in flight_data['Aircraft type']]

In [6]:
# list of numerical and categorical features to select 
l_numeric_cols = [ 'DEP_HOUR', 'temp', 'dwpt', 'rhum', 'prcp', 'wspd', 'pres', 'coco']
l_categorical_cols = ['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'aircraft_type_new']
target = "target"

In [7]:
# converting the categorical columns to categorical
for col in l_categorical_cols:
    flight_data[col] = flight_data[col].astype('category')

In [8]:
flight_data = flight_data[l_numeric_cols + l_categorical_cols + [target] + ['YEAR']]

In [9]:
# getting dummu values for categorical columns
QUARTER_dummies = pd.get_dummies(flight_data['QUARTER'], prefix='QUARTER', drop_first=True)
MONTH_dummies = pd.get_dummies(flight_data['MONTH'], prefix='MONTH', drop_first=True)
DAY_OF_MONTH_dummies = pd.get_dummies(flight_data['DAY_OF_MONTH'], prefix='DAY_OF_MONTH', drop_first=True)
DAY_OF_WEEK_dummies = pd.get_dummies(flight_data['DAY_OF_WEEK'], prefix='DAY_OF_WEEK', drop_first=True)
Aircraft_type_dummies = pd.get_dummies(flight_data['aircraft_type_new'], prefix='Aircraft_type', drop_first=True)

In [10]:
# Aircraft_type_dummies.columns = ['Aircraft_{}'.format(i) for i in range(len(Aircraft_type_dummies.columns))]

In [11]:
flight_data = flight_data.drop(l_categorical_cols, axis=1)

In [12]:
dummy_flight_data = pd.concat([flight_data, QUARTER_dummies, MONTH_dummies,
                               DAY_OF_MONTH_dummies, DAY_OF_WEEK_dummies, Aircraft_type_dummies ], axis=1)

In [13]:
del Aircraft_type_dummies,QUARTER_dummies,MONTH_dummies,DAY_OF_MONTH_dummies,DAY_OF_WEEK_dummies

In [14]:
train = dummy_flight_data[dummy_flight_data['YEAR'] == 2019]
test = dummy_flight_data[dummy_flight_data['YEAR'] == 2020]

In [15]:
del flight_data, dummy_flight_data

In [16]:
train.target.value_counts()

0    6500083
1     903698
2     317943
3     215739
Name: target, dtype: int64

In [17]:
# from imblearn.combine import SMOTEENN
# sm = SMOTEENN(random_state=42)
# X_res, y_res = sm.fit_resample(train.drop(["YEAR","target"], axis=1).values, train[target])

from imblearn.under_sampling import RandomUnderSampler

undersample = RandomUnderSampler(sampling_strategy='majority', random_state=42)
X_res, y_res = undersample.fit_resample(train.drop(["YEAR","target"], axis=1).values,train[target])

In [18]:
del train

In [19]:
y_res.value_counts()

1    903698
2    317943
0    215739
3    215739
Name: target, dtype: int64

In [20]:
X_res1, y_res1 = undersample.fit_resample(X_res, y_res)
del X_res, y_res

In [21]:
y_res1.value_counts()

2    317943
0    215739
1    215739
3    215739
Name: target, dtype: int64

In [22]:
X_res2, y_res2 = undersample.fit_resample(X_res1, y_res1)
del X_res1, y_res1

In [23]:
y_res2.value_counts()

0    215739
1    215739
2    215739
3    215739
Name: target, dtype: int64

In [25]:
train = pd.DataFrame(X_res2, columns=test.columns.drop(["YEAR","target"]))

In [26]:
train['target'] = y_res2

In [8]:
# train.to_csv('undersampled_training_data_2019.csv', index=False)
train = pd.read_csv('undersampled_training_data_2019.csv')

In [28]:
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(
    train.drop(["target"], axis=1).values,
     train[target]
     )

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)

In [29]:
path = "./models"
filename = "2019train_RF_undersample.mdl"

In [30]:
with open(os.path.join(path, filename), "wb") as f:
    pickle.dump(classifier, f, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
classifier = pickle.load(open("./models/2019train_RF_undersample.mdl",'rb'))

C:\anaconda\envs\python_37\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
C:\anaconda\envs\python_37\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [9]:
feature_names = train.columns.drop('target')

In [11]:
importances = pd.DataFrame(
  pd.Series(dict(zip(feature_names, classifier.feature_importances_)))
).assign(
  normalized=lambda df: df / df.sum()
).sort_values(
  by="normalized", ascending=False
)

In [12]:
importances[["normalized"]].head(10)

,normalized
DEP_HOUR,0.160344
pres,0.073252
temp,0.065634
rhum,0.060494
dwpt,0.058674
wspd,0.044816
coco,0.031303
Aircraft_type_Embraer E175LR,0.031270
Aircraft_type_Airbus A319,0.030515
Aircraft_type_Airbus A321,0.030317


In [31]:
# predicting the test results 
y_pred = classifier.predict(test.drop(["YEAR","target"], axis=1).values)

In [32]:
print(confusion_matrix(test[target],y_pred))

[[2224392  896544  529142  654305]
 [ 102261   70914   44451   59432]
 [  27310   17861   15839   24456]
 [  15910    9561    9867   18988]]


In [33]:
print(classification_report(test[target],y_pred))

              precision    recall  f1-score   support

           0       0.94      0.52      0.67   4304383
           1       0.07      0.26      0.11    277058
           2       0.03      0.19      0.05     85466
           3       0.03      0.35      0.05     54326

    accuracy                           0.49   4721233
   macro avg       0.27      0.33      0.22   4721233
weighted avg       0.86      0.49      0.62   4721233



In [34]:
print("Testing  set score for model: %f" % classifier.score(test.drop(["YEAR","target"], axis=1).values  , test[target] ))

Testing  set score for model: 0.493543
